# Klasifikasi Emosi Audio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PSD A/audio

Mounted at /content/drive
/content/drive/MyDrive/PSD A/audio


In [ ]:
%%capture
pip install librosa

**Dataset Understanding**

Dataset ini digunakan untuk melatih model klasifikasi emosi dalam audio, dengan fokus pada tujuh emosi utama. Dataset ini terdiri dari rekaman audio yang dibuat dengan mengucapkan 200 kata target dalam frasa pembawa "Katakan kata _" oleh dua aktris berusia 26 dan 64 tahun.

Setiap aktris merekam frasa tersebut dengan mengekspresikan tujuh emosi utama: marah, jijik, takut, bahagia, kejutan menyenangkan, kesedihan, dan netral. Totalnya, terdapat 2800 data audio dalam format WAV.

Dataset ini diatur dengan baik, di mana setiap aktris dan emosi yang mereka ekspresikan ditempatkan dalam folder tersendiri. Di dalam setiap folder, terdapat file audio untuk setiap kata target yang diucapkan.

Format file audio yang digunakan adalah WAV, yang merupakan format umum untuk menyimpan data audio tanpa kehilangan kualitas.

Dengan penyusunan dataset seperti ini, diharapkan dapat memudahkan proses pelatihan model untuk mengenali dan mengklasifikasikan emosi dari rekaman audio yang diberikan.

**Ektraksi Fitur**

In [ ]:
# Import Library
import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode

In [ ]:
folders=['YAF_sad','YAF_pleasant_surprised','YAF_neutral',
         'YAF_happy','YAF_fear','YAF_disgust','YAF_angry',
         'OAF_Sad','OAF_Pleasant_surprise','OAF_neutral',
         'OAF_happy','OAF_Fear','OAF_disgust',
         'OAF_angry',
         ]

In [ ]:
def calculate_statistics(audio_path):
    y, sr = librosa.load(audio_path)

    # UNTUK MENGHITUNG NILAI STATISTIKA
    mean = np.mean(y)
    std_dev = np.std(y)
    max_value = np.max(y)
    min_value = np.min(y)
    median = np.median(y)
    skewness = skew(y)  # Calculate skewness
    kurt = kurtosis(y)  # Calculate kurtosis
    q1 = np.percentile(y, 25)
    q3 = np.percentile(y, 75)
    mode_value, _ = mode(y)  # Calculate mode
    iqr = q3 - q1

    # UNTUK MENGHITUNG NILAI ZCR
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])

    # UNTUK MENGHITUNG NILAI RMSE
    rmse = np.sum(y**2) / len(y)
    rmse_median = np.median(y**2)
    rmse_std_dev = np.std(y**2)
    rmse_kurtosis = kurtosis(y**2)
    rmse_skew = skew(y**2)

    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew, rmse, rmse_median, rmse_std_dev, rmse_kurtosis, rmse_skew]

**Penjelasan Rumus**

1. Mean : nilai tengah dari suatu data set

2. Standar Deviasi : mengukur seberapa tersebar atau jauh nilai-nilai individu dalam dataset dari nilai rata-ratanya.

3. Median : nilai tengah dari suatu set data yang telah diurutkan.

4. Skewness : mengukur sejauh mana distribusi data miring dari simetri.

5. Kurtosis : mengukur tingkat kecuraman (tumpukan atau landasan) distribusi data.

6. Zero Crossing Rate : mengukur seberapa sering sinyal melintasi nilai nol per detik.

7. Root Mean Square Error : mengukur rata-rata perbedaan antara nilai aktual dan nilai yang diprediksi (dalam hal ini, amplitudo sinyal audio).

In [ ]:
# Membuat DataFrame untuk menyimpan hasil
data = []

In [ ]:
# Loop melalui setiap folder dan file audio
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            data.append([folder] + statistics)

In [ ]:
# Membuat DataFrame dari data
columns =  ['Label'] + ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']
df = pd.DataFrame(data, columns=columns)

In [ ]:
# Menampilkan file CSV
df

,Label,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,YAF_sad,0.128581,0.033203,0.225249,3.231511,2.213080,0.001907,0.000391,0.004361,28.695116,4.718993
1,YAF_sad,0.097222,0.021484,0.175901,6.527142,2.770304,0.000423,0.000087,0.000826,15.044502,3.552011
2,YAF_sad,0.138011,0.057129,0.193587,3.628069,2.167333,0.001019,0.000084,0.002397,23.264337,4.194062
3,YAF_sad,0.106989,0.027100,0.183249,5.155950,2.518261,0.000528,0.000045,0.001077,12.974666,3.380200
4,YAF_sad,0.124418,0.030762,0.203419,3.494584,2.197851,0.000502,0.000045,0.001103,20.908828,4.065221
...,...,...,...,...,...,...,...,...,...,...,...
2808,OAF_angry,0.092861,0.052979,0.097337,3.818796,2.091882,0.002065,0.000113,0.005112,47.387673,5.346160
2809,OAF_angry,0.095171,0.054932,0.117568,4.654374,2.418914,0.003045,0.000289,0.007432,27.448332,4.589201
2810,OAF_angry,0.084954,0.051270,0.109765,4.836171,2.427723,0.001707,0.000135,0.004266,25.751815,4.479781
2811,OAF_angry,0.080717,0.057373,0.088056,5.404383,2.449666,0.001482,0.000140,0.004272,52.804723,6.344142


In [ ]:
# Menyimpan DataFrame ke dalam file CSV
df.to_csv('hasil_statistik_audio.csv', index=False)

**Preprocessing Menggunakan Z-Score**

Normalisasi menggunakan Z-score atau biasa disebut Standarisasi yaitu merupakan teknik untuk mengubah nilai dalam data sehingga memiliki rata-rata 0 dan standar deviasi 1. Standarisasi membantu dalam mengubah distribusi data menjadi distribusi normal dan membandingkan variabel-variabel dengan unit yang berbeda. Z-Score Scalling digunakan untuk mengubah nilai dalam data dengan mengukur sejauh mana nilai dari rata-rata dalam satuan standar deviasinya.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Baca file CSV
norm = pd.read_csv("hasil_statistik_audio.csv")

In [ ]:
# Tentukan kolom yang akan distandarisasi
kolom = ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']


In [ ]:
# Inisialisasi StandardScaler
scaler = StandardScaler()
# Lakukan standarisasi pada kolom yang telah ditentukan
norm[kolom] = scaler.fit_transform(norm[kolom])
# Simpan DataFrame yang telah distandarisasi ke dalam file CSV baru
norm.to_csv("normalisasi.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [ ]:
# Baca data dari file CSV
dataknn= pd.read_csv('hasil_statistik_audio.csv')
# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label'], axis=1)  # Ganti 'target_column' dengan nama kolom target
y = dataknn['Label']
# split data into train and test sets
X_train,X_test,y_train, y_test= train_test_split(X, y, random_state=1, test_size=0.2)
# define scaler
scaler = StandardScaler()
# fit scaler on the training dataset
scaler.fit(X_train)
# save the scaler
dump(scaler, open('skala.pkl', 'wb'))
# transform the training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import pickle
with open('skala.pkl', 'rb') as standarisasi:
    loadscal= pickle.load(standarisasi)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

**Modelling Tanpa PCA**

In [ ]:
K = 50
acc = np.zeros((K-1))

for n in range(1,K,2):
    knn = KNeighborsClassifier(n_neighbors= n, metric = "euclidean").fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)

    acc[n-1]= accuracy_score(y_test,y_pred)

print('akurasi terbaik adalah ', acc.max(), 'dengan nilai k =', acc.argmax()+1)

akurasi terbaik adalah  0.7246891651865008 dengan nilai k = 5


In [ ]:
knn = KNeighborsClassifier(n_neighbors= 7, metric = "euclidean")
#dump(knn, open('modelknn.pkl', 'wb'))

In [ ]:
#import pickle
#with open('modelknn.pkl', 'rb') as knn:
    # loadknn= pickle.load(knn)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [ ]:
y_pred = knn.predict(X_test_scaled)
y_pred

array(['OAF_happy', 'YAF_fear', 'YAF_disgust', 'YAF_happy',
       'YAF_pleasant_surprised', 'OAF_Pleasant_surprise', 'YAF_neutral',
       'YAF_sad', 'OAF_neutral', 'YAF_sad', 'YAF_pleasant_surprised',
       'OAF_disgust', 'YAF_angry', 'OAF_Fear', 'OAF_Pleasant_surprise',
       'OAF_Pleasant_surprise', 'OAF_disgust', 'OAF_happy', 'OAF_Sad',
       'YAF_happy', 'OAF_Fear', 'OAF_Fear', 'OAF_neutral', 'YAF_sad',
       'YAF_sad', 'OAF_Sad', 'YAF_angry', 'YAF_sad', 'OAF_Sad',
       'YAF_pleasant_surprised', 'OAF_Sad', 'OAF_Sad', 'OAF_Sad',
       'YAF_fear', 'OAF_disgust', 'OAF_Fear', 'YAF_pleasant_surprised',
       'YAF_happy', 'OAF_Pleasant_surprise', 'YAF_neutral', 'YAF_angry',
       'OAF_Sad', 'OAF_Fear', 'OAF_Sad', 'YAF_happy', 'OAF_neutral',
       'OAF_Pleasant_surprise', 'YAF_angry', 'YAF_angry', 'OAF_Fear',
       'YAF_happy', 'OAF_happy', 'YAF_neutral', 'YAF_disgust',
       'YAF_angry', 'YAF_angry', 'OAF_Sad', 'OAF_neutral', 'OAF_Sad',
       'YAF_pleasant_surprised', 'OAF

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
print("Akurasi:",accuracy)

Akurasi: 0.7069271758436945


In [ ]:
dump(knn, open('knn.pkl', 'wb'))

**Reduksi Data Menggunakan PCA**

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=9)
pca.fit(X_train_scaled)
pca_train = pca.transform(X_train_scaled)

In [ ]:
pca_train.shape

(2250, 9)

In [ ]:
pca_test = pca.transform(X_test_scaled)

In [ ]:
type(pca_test)

numpy.ndarray

In [ ]:
pca_test.shape

(563, 9)

In [ ]:
dump(pca_train, open('PCA9.pkl', 'wb'))

In [ ]:
import pickle
with open('PCA9.pkl', 'rb') as pca:
    loadpca= pickle.load(pca)

**Modelling KNN Menggunakan PCA**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=11)
classifier.fit(pca_train, y_train)

KNeighborsClassifier(n_neighbors=11)

In [ ]:
y_prediksi = classifier.predict(pca_test)
y_prediksi

array(['OAF_happy', 'YAF_fear', 'YAF_disgust', 'YAF_happy', 'YAF_disgust',
       'OAF_Pleasant_surprise', 'YAF_neutral', 'YAF_sad', 'OAF_neutral',
       'YAF_sad', 'YAF_disgust', 'OAF_disgust', 'YAF_angry', 'OAF_Fear',
       'OAF_happy', 'OAF_Pleasant_surprise', 'OAF_disgust', 'OAF_happy',
       'OAF_Sad', 'YAF_happy', 'OAF_Fear', 'OAF_Fear', 'OAF_neutral',
       'YAF_sad', 'YAF_sad', 'OAF_Sad', 'YAF_angry', 'YAF_sad', 'OAF_Sad',
       'YAF_pleasant_surprised', 'OAF_Sad', 'OAF_Sad', 'OAF_Sad',
       'YAF_fear', 'OAF_disgust', 'OAF_disgust', 'YAF_pleasant_surprised',
       'YAF_happy', 'OAF_Pleasant_surprise', 'YAF_neutral', 'YAF_fear',
       'OAF_Sad', 'OAF_happy', 'OAF_Sad', 'YAF_happy', 'OAF_neutral',
       'OAF_Fear', 'YAF_angry', 'YAF_angry', 'OAF_Fear', 'YAF_happy',
       'OAF_happy', 'YAF_neutral', 'YAF_disgust', 'YAF_angry', 'YAF_fear',
       'OAF_Sad', 'OAF_neutral', 'OAF_Sad', 'YAF_pleasant_surprised',
       'OAF_disgust', 'OAF_happy', 'OAF_happy', 'YAF_angry',
  

In [ ]:
acc_pca= accuracy_score(y_test,y_prediksi)
print("Akurasi:",acc_pca)
#Akurasi: 0.7153024911032029

Akurasi: 0.7158081705150977


**Aplikasi Streamlit :** [klik disini](https://uts-psd-a-robiatuladawiyah-210411100187.streamlit.app/)